# Explore here

It's recommended to use this notebook for exploration purposes.

In [29]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
import requests
import time
import lxml
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
# load the .env file variables
load_dotenv()

url = 'https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue'

# Petición para descargar el fichero de Internet
response = requests.get(url, time.sleep(10)).text
if "403 Forbidden" in response:
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"}
    request = requests.get(url, headers = headers)
    time.sleep(10)
    response = request.text



soup = bs(response, 'html.parser')

revenues = soup.find_all('table', class_="historical_data_table")[0]

revenues



# 1) Connect to the database here using the SQLAlchemy's create_engine function

# 2) Execute the SQL sentences to create your tables using the SQLAlchemy's execute function

# 3) Execute the SQL sentences to insert your data using the SQLAlchemy's execute function

# 4) Use pandas to print one of the tables as dataframes using read_sql function


<table class="historical_data_table table">
<thead>
<tr>
<th colspan="2" style="text-align:center">Tesla Annual Revenue<br/><span style="font-size:14px;">(Millions of US $)</span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:center">2022</td>
<td style="text-align:center">$81,462</td>
</tr>
<tr>
<td style="text-align:center">2021</td>
<td style="text-align:center">$53,823</td>
</tr>
<tr>
<td style="text-align:center">2020</td>
<td style="text-align:center">$31,536</td>
</tr>
<tr>
<td style="text-align:center">2019</td>
<td style="text-align:center">$24,578</td>
</tr>
<tr>
<td style="text-align:center">2018</td>
<td style="text-align:center">$21,461</td>
</tr>
<tr>
<td style="text-align:center">2017</td>
<td style="text-align:center">$11,759</td>
</tr>
<tr>
<td style="text-align:center">2016</td>
<td style="text-align:center">$7,000</td>
</tr>
<tr>
<td style="text-align:center">2015</td>
<td style="text-align:center">$4,046</td>
</tr>
<tr>
<td style="text-align:center">2014</t

In [40]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
import requests
import time
import lxml
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
list1 = []
list2 = []
rows = revenues.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    if len(cols) == 0:
        continue
    else:
        list1.append(cols[0].text)
        list2.append(cols[1].text)

print(list1)


['2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009']


In [31]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
import requests
import time
import lxml
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen


for row in rows:
    revenues = revenues + "<tr>"
    for col in row:
        revenues = revenues + "<td>" + col.replace(" ", "") + "</td>"
    revenues = revenues + "</tr>"
    
revenues = revenues + "</table>"


TypeError: unsupported operand type(s) for +: 'Tag' and 'str'